In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
from solar_power.environment_prediction_model import SolarPredictionModelExtended 
from solar_power.solar_prediction_model import FeatureSolarPredictionModel
from electrical_load.load_prediction_model import BuildingEnergyModel

# * 모델 선언
# * 랜덤 시드 값을 모델 간 동일하게 42로 설정
# * 테스트 데이터 개수는 모델 간 동일하게 12 설정, 12/62 = 19.35%

random_state = 42
test_size = 12 

# * 기상 데이터 to 환경 감시 데이터 예측 모델
env_pred_model = SolarPredictionModelExtended("./solar_power/merged_result.csv", random_state=random_state, test_size=test_size)
env_pred_model.load_data()
env_pred_model.hyperparameter_tuning()
env_pred_model.train_model()

# * 환경 감시 데이터 to 태양광 발전량 예측 모델
solar_pred_model = FeatureSolarPredictionModel("./solar_power/merged_result.csv", random_state=random_state, test_size=test_size)
solar_pred_model.load_data()
solar_pred_model.feature_selection()
solar_pred_model.train_model()

# * 건물 전기 부하 예측 모델
energy_pred_model = BuildingEnergyModel("./electrical_load/merged_data.csv", random_state=random_state, test_size=test_size)
energy_pred_model.load_data()
energy_pred_model.train_model()

In [15]:

# * 태양광 발전량 예측 모델에 입력할 정규화된 입력 데이터
# * test_x 는 1시간 단위의 데이터
# * test_date 는 예측할 시간의 날짜

import numpy as np
rand_int = np.random.randint(0, len(env_pred_model.X_test))

test_x = env_pred_model.X_test.iloc[rand_int:rand_int+1, :]
test_index = test_x.index[0]
test_date = env_pred_model.data.loc[test_index, 'day']

test_x

,강수확률,일최저기온,하늘상태,일최고기온,습도,풍향,풍속,time
1159,0.2,1.0,0.666667,0.571429,0.916667,0.620961,0.112903,0.652174


In [16]:

# * 1시간 단위의 데이터와 같은 날짜의 데이터 24개를 합쳐 same_day_X_test 생성

same_day_data = env_pred_model.data[env_pred_model.data['day'] == test_date]
same_day_data_indexes = same_day_data.index
same_day_X_test = env_pred_model.X_test.loc[env_pred_model.X_test.index.isin(same_day_data_indexes)]

same_day_X_test

,강수확률,일최저기온,하늘상태,일최고기온,습도,풍향,풍속,time
1152,0.2,1.0,0.666667,0.571429,0.833333,0.387524,0.129032,0.500000
1153,0.2,1.0,0.666667,0.571429,0.916667,0.206107,0.129032,0.521739
1154,0.2,1.0,0.666667,0.571429,0.916667,0.345492,0.129032,0.543478
1155,0.2,1.0,0.666667,0.571429,0.916667,0.146447,0.145161,0.565217
1156,0.2,1.0,0.666667,0.571429,0.916667,0.220404,0.161290,0.586957
1157,0.2,1.0,0.666667,0.571429,0.916667,0.328990,0.177419,0.608696
1158,0.2,1.0,0.666667,0.571429,0.916667,0.456422,0.161290,0.630435
1159,0.2,1.0,0.666667,0.571429,0.916667,0.620961,0.112903,0.652174
1160,0.2,1.0,0.666667,0.571429,0.833333,0.620961,0.112903,0.673913
1161,0.3,1.0,1.000000,0.571429,0.750000,0.465122,0.193548,0.695652


# 태양광 발전량 예측

In [17]:
import pandas as pd
# * 예측한 태양광 발전량 데이터

res_solar = env_pred_model.predict(same_day_X_test) # 기상 데이터 -> 환경감시 데이터
res_solar = solar_pred_model.predict(res_solar) # 환경감시 데이터 -> 태양광 발전량 데이터
res_solar

,축구장,학생회관,중앙창고,학사과정,다산빌딩,시설관리동,대학C동,동물실험동,중앙도서관,LG도서관,신재생에너지동,삼성환경동,중앙연구기기센터,산업협력관,기숙사 B동
0,2.7,-0.1,0.5,-0.4,-0.1,0.3,0.1,0.5,1.0,1.0,-0.7,-0.1,0.1,1.4,1.8
1,1.8,-0.3,0.3,-0.4,-0.1,0.0,0.1,0.3,0.7,0.8,-0.8,-0.2,-0.1,0.9,1.4
2,2.3,-0.2,0.4,-0.4,-0.1,0.2,0.1,0.4,0.9,0.9,-0.7,-0.2,0.0,1.2,1.6
3,0.9,-0.6,0.1,-0.4,-0.1,-0.3,0.0,0.1,0.4,0.5,-0.9,-0.3,-0.2,0.3,1.0
4,1.7,-0.4,0.3,-0.4,-0.1,-0.0,0.1,0.3,0.7,0.7,-0.8,-0.2,-0.1,0.8,1.4
5,2.1,-0.2,0.4,-0.4,-0.1,0.1,0.1,0.4,0.8,0.8,-0.7,-0.2,0.0,1.1,1.5
6,2.6,-0.1,0.5,-0.4,-0.1,0.3,0.1,0.5,1.0,1.0,-0.7,-0.1,0.1,1.4,1.7
7,2.0,4.9,4.3,3.0,0.4,8.0,1.1,1.5,0.6,0.7,2.2,2.7,2.1,6.4,3.6
8,18.6,11.1,7.6,5.9,5.9,21.2,2.9,7.2,5.2,6.9,12.6,11.5,18.7,21.0,9.1
9,32.4,14.0,8.4,7.2,10.4,28.1,4.3,12.3,9.1,12.2,20.6,18.0,32.9,35.1,14.2


In [18]:
# * 실제 태양광 발전량

same_day_y_test = solar_pred_model.y_test.loc[solar_pred_model.y_test.index.isin(same_day_data_indexes)]
same_day_y_test

,축구장,학생회관,중앙창고,학사과정,다산빌딩,시설관리동,대학C동,동물실험동,중앙도서관,LG도서관,신재생에너지동,삼성환경동,중앙연구기기센터,산업협력관,기숙사 B동
1152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1153,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1158,0.0,0.0,0.0,0.0,-0.2,0.0,0.0,0.1,0.0,0.0,0.6,0.0,0.0,0.3,0.0
1159,4.7,3.2,1.8,1.6,2.0,5.1,1.0,2.8,1.9,2.7,5.0,4.0,7.3,6.6,3.3
1160,8.0,6.5,4.1,3.0,5.2,11.3,2.1,5.3,3.9,4.9,9.4,9.3,14.5,12.9,6.0
1161,28.8,20.2,15.2,10.8,12.1,33.4,5.4,15.9,12.1,15.1,26.8,23.4,43.6,45.4,19.5


# 전기 부하량 예측

In [19]:
# * 부하량 예측 모델에 입력할 정규화된 입력 데이터

same_day_X_test = energy_pred_model.X_test.loc[energy_pred_model.X_test.index.isin(same_day_data_indexes)]
same_day_X_test

,time,1시간기온,1시간강수량,일최고기온,일최저기온,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
1152,0,23.0,0.0,31.0,26.0,False,False,False,False,True,False,False
1153,1,23.0,0.0,31.0,26.0,False,False,False,False,True,False,False
1154,2,22.0,0.0,31.0,26.0,False,False,False,False,True,False,False
1155,3,22.0,0.0,31.0,26.0,False,False,False,False,True,False,False
1156,4,21.0,0.0,31.0,26.0,False,False,False,False,True,False,False
1157,5,21.0,0.0,31.0,26.0,False,False,False,False,True,False,False
1158,6,21.0,0.0,31.0,26.0,False,False,False,False,True,False,False
1159,7,22.0,0.0,31.0,26.0,False,False,False,False,True,False,False
1160,8,23.0,0.0,31.0,26.0,False,False,False,False,True,False,False
1161,9,25.0,0.0,31.0,26.0,False,False,False,False,True,False,False


In [20]:
# * 예측한 전기 부하량

res_load = energy_pred_model.predict(same_day_X_test)
res_load

,중앙P/P_석사,전기전자컴퓨터공학동_석사,신소재공학_석사,생명과학동_석사,기계공학동_석사,LG도서관_석사,창업B동_석사,금호관_석사,"냉각수순환펌프1,2,3_석사","냉각수순환펌프4,5,6_석사",...,대학B동(E)_학사,대학기숙사A동(E)_학사,학사P/P(E)_학사,제2학생회관(E)_학사,교원아파트(E)_학사,대학C동(E)_학사,중앙연구기기센터(E)_학사,대학A동(E)_학사,다산빌딩(E)_학사,산학협력연구동(E)_학사
0,152.2,421.8,404.9,473.5,113.4,237.4,34.9,50.8,0.0,0.0,...,32.0,40.2,41.6,26.0,23.6,19.8,51.3,16.5,52.6,18.1
1,153.3,419.3,398.6,455.1,111.9,223.2,29.9,50.6,0.0,0.0,...,29.4,39.8,40.3,26.8,23.9,17.3,50.6,15.4,50.2,18.1
2,140.7,406.7,382.8,428.2,104.6,221.0,30.0,50.3,0.0,0.0,...,26.7,31.0,41.7,24.5,18.2,16.8,49.5,14.3,43.9,18.4
3,143.2,404.4,380.4,422.5,103.7,219.0,30.0,50.0,0.0,0.0,...,25.7,28.1,44.3,23.4,15.3,16.5,48.0,13.7,41.5,18.1
4,145.2,404.7,377.2,426.6,103.4,216.6,27.9,49.6,0.0,0.0,...,24.0,26.4,37.3,22.6,17.0,15.7,48.1,12.8,41.0,18.2
5,127.3,407.1,385.5,425.3,103.0,218.1,35.3,51.9,0.0,0.0,...,31.3,29.1,33.9,28.2,22.2,16.6,50.4,15.0,41.1,18.2
6,107.8,404.7,385.3,451.0,104.8,225.9,32.9,50.8,0.0,0.0,...,34.4,28.2,27.3,33.0,25.4,16.7,49.3,18.5,42.1,20.6
7,109.7,407.7,386.0,450.5,115.8,245.5,34.5,52.8,9.4,6.4,...,37.4,29.8,27.9,33.1,29.4,18.4,54.8,21.1,44.5,20.6
8,142.9,452.3,436.4,541.7,192.5,292.4,50.0,68.1,201.3,189.4,...,46.2,40.5,36.7,43.2,40.3,32.0,63.9,28.2,67.5,25.4
9,150.8,498.0,520.9,649.3,229.0,337.9,56.8,71.6,204.6,114.6,...,52.7,41.8,29.1,46.5,37.2,35.5,68.0,28.7,78.9,25.1


In [21]:
# * 실제 전기 부하량

same_day_y_test = energy_pred_model.y_test.loc[energy_pred_model.y_test.index.isin(same_day_data_indexes)]
same_day_y_test

,중앙P/P_석사,전기전자컴퓨터공학동_석사,신소재공학_석사,생명과학동_석사,기계공학동_석사,LG도서관_석사,창업B동_석사,금호관_석사,"냉각수순환펌프1,2,3_석사","냉각수순환펌프4,5,6_석사",...,대학B동(E)_학사,대학기숙사A동(E)_학사,학사P/P(E)_학사,제2학생회관(E)_학사,교원아파트(E)_학사,대학C동(E)_학사,중앙연구기기센터(E)_학사,대학A동(E)_학사,다산빌딩(E)_학사,산학협력연구동(E)_학사
1152,157.3,409.8,412.1,422.1,108.3,233.5,20.5,50.1,0.0,0.0,...,37.8,47.3,38.2,33.0,29.4,31.7,48.7,14.2,62.5,18.2
1153,139.1,400.2,408.7,409.8,109.2,229.5,30.0,49.7,0.0,0.0,...,37.6,36.6,38.6,34.2,31.2,17.2,47.6,16.7,60.1,18.7
1154,131.5,399.9,405.5,461.0,110.0,235.9,26.7,49.7,0.0,0.0,...,25.6,38.7,36.6,21.2,18.7,30.0,48.0,14.8,41.3,18.4
1155,132.0,400.9,401.8,465.8,103.7,223.5,28.1,50.7,0.0,0.0,...,25.5,22.5,63.7,30.5,25.1,16.9,47.1,15.2,41.3,18.1
1156,147.5,398.6,376.7,416.5,98.2,185.8,32.1,49.7,0.0,0.0,...,23.6,29.1,47.9,21.1,14.4,29.0,47.7,12.8,49.1,18.3
1157,133.9,392.5,366.6,406.6,99.0,180.3,25.8,49.6,0.0,0.0,...,27.5,22.4,35.3,26.8,21.7,23.2,47.1,15.7,48.1,17.9
1158,105.4,392.1,370.7,426.9,104.1,202.5,27.3,50.9,0.0,0.0,...,39.2,18.9,22.2,31.4,18.0,15.7,47.4,14.6,49.6,21.1
1159,102.7,392.3,377.5,499.0,116.8,249.0,53.7,52.0,223.8,139.4,...,43.2,30.7,24.4,32.3,32.2,27.2,59.5,17.2,56.9,21.1
1160,163.6,430.2,420.1,549.8,165.0,298.4,37.8,76.0,210.7,138.9,...,45.6,39.1,42.5,33.6,42.1,32.1,68.9,25.8,64.9,23.4
1161,153.7,478.8,473.4,553.4,192.8,324.3,60.6,70.3,210.4,138.2,...,51.5,36.7,26.8,49.2,32.3,31.3,68.9,17.1,81.8,22.9


In [22]:
import pandas as pd
res_solar_sum = pd.DataFrame(res_solar.sum(axis=1), columns=['Total Predicted Solar'])
res_load_sum = pd.DataFrame(res_load.sum(axis=1), columns=['Total Predicted Load'])

# 전기 요금 예측 알고리즘

In [23]:
# * 예측 발전량과 부하량을 통해 전기 요금 산출

from algorithm.schedule_algorithm import schedule

algorithm = schedule()

res_solar_sum = res_solar_sum["Total Predicted Solar"].values.tolist()
res_load_sum = res_load_sum["Total Predicted Load"].values.tolist()
result = algorithm.schedule_optimizer([res_solar_sum],[res_load_sum])


In [24]:
# * 원으로 변경 및 자리수 맞추기

integer_result = int(result)

integer_result_= int(integer_result / 10) * 10
final_result = "{:,}".format(integer_result)
print(final_result+"원")

24,035,074원
